In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7292,application_1589299642358_1788,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_final"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
# Initialization
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7187,application_1589299642358_1683,pyspark,idle,Link,Link,
7244,application_1589299642358_1740,pyspark,idle,Link,Link,
7269,application_1589299642358_1765,pyspark,idle,Link,Link,
7271,application_1589299642358_1767,pyspark,idle,Link,Link,
7272,application_1589299642358_1768,pyspark,idle,Link,Link,
7273,application_1589299642358_1769,pyspark,idle,Link,Link,
7274,application_1589299642358_1770,pyspark,idle,Link,Link,
7275,application_1589299642358_1771,pyspark,idle,Link,Link,
7276,application_1589299642358_1772,pyspark,idle,Link,Link,
7279,application_1589299642358_1775,pyspark,idle,Link,Link,


In [116]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
sbb.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
|betriebstag|   fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|ankunftszeit|an_prognose|an_prognose_status|    abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
| 03.09.2018|80:06____:17010:000|   80:06____|           DB|   DB Regio AG|       Zug|    17010|    

In [118]:
sbb = (
    sbb
    .where(sbb.zusatzfahrt_tf == 'false')
    .where(sbb.faellt_aus_tf == 'false')
    .where(sbb.durchfahrt_tf == 'false')
    .where(sbb.ankunftszeit.isNotNull())
    .where(sbb.an_prognose.isNotNull())
    .where(sbb.an_prognose_status.isin('REAL', 'PROGNOSE'))
    
    .withColumn('ankunftszeit_ts', f.unix_timestamp(f.col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('an_prognose_ts', f.unix_timestamp(f.col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long'))
    .withColumn('delay', f.col('an_prognose_ts') - f.col('ankunftszeit_ts'))
    
    .select(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status', 'delay'])
    .dropDuplicates(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status'])
).persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
delays = (
    sbb
    .groupBy(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit'])
    .agg(
        f.first(f.when(sbb.an_prognose_status == 'REAL', sbb.delay).otherwise(None)).alias('real_delay'),
        f.first(f.when(sbb.an_prognose_status == 'PROGNOSE', sbb.delay).otherwise(None)).alias('prognose_delay'),
    )
    .withColumn('delay', f.when(f.col('real_delay').isNotNull(), f.col('real_delay')).otherwise(f.col('prognose_delay')))
    .withColumn('is_delayed', f.when(f.col('delay') > 0, 1).otherwise(0))
)

delays.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------+----------------+----------+--------------+-----+----------+
|betriebstag|    fahrt_bezeichner|  bpuic|    ankunftszeit|real_delay|prognose_delay|delay|is_delayed|
+-----------+--------------------+-------+----------------+----------+--------------+-----+----------+
| 01.01.2018|85:3849:62923-050...|8591373|01.01.2018 16:48|      null|            42|   42|         1|
| 01.01.2018|85:823:210795-154...|8500160|01.01.2018 19:14|      null|            17|   17|         1|
| 01.01.2018|85:836:109741-000...|8573460|01.01.2018 19:39|      null|           115|  115|         1|
| 01.01.2018|85:836:109995-000...|8573465|01.01.2018 18:13|      null|            48|   48|         1|
| 01.01.2018|85:849:174517-253...|8590805|                |      null|          null| null|         0|
| 01.01.2018|85:849:65208-05032-1|8591101|01.01.2018 15:15|      null|            88|   88|         1|
| 01.01.2018|85:849:66143-05040-1|8591290|01.01.2018 09:58|      null|   

In [120]:
probability = (
    delays
    .groupBy(['fahrt_bezeichner', 'bpuic', 'ankunftszeit'])
    .agg(
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
probability.write.format("orc").save("/user/gflueck/delay_analysis.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…